In [1]:
import csv
import numpy as np
import os
import pandas as pd
import scipy.interpolate
import sklearn.metrics
import sys
sys.path.append("../src")
import localmodule


if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO


from matplotlib import pyplot as plt
%matplotlib inline

# Define constants.
dataset_name = localmodule.get_dataset_name()
models_dir = localmodule.get_models_dir()
units = localmodule.get_units()
n_units = len(units)
n_trials = 10


In [10]:
# Define model directory
model_name = "icassp-convnet"
aug_kind_str = "none"
if not aug_kind_str == "none":
    model_name = "_".join([model_name, "aug-" + aug_kind_str])
model_dir = os.path.join(models_dir, model_name)

fns, fps, tns, tps, accs = [], [], [], [], []

for test_unit_id in range(6):

    test_unit_str = units[test_unit_id]
    test_unit_dir = os.path.join(model_dir, test_unit_str)

    val_losses = []
    for trial_id in range(n_trials):
        trial_str = "trial-" + str(trial_id)
        trial_dir = os.path.join(test_unit_dir, trial_str)
        history_name = "_".join([
            dataset_name,
            model_name,
            test_unit_str,
            trial_str,
            "history.csv"
        ])
        history_path = os.path.join(
            trial_dir, history_name)
        try:
            history_df = pd.read_csv(history_path)
            val_loss = min(history_df["Validation loss"])
        except:
            val_loss = np.inf
        val_losses.append(val_loss)

    val_losses = np.array(val_losses)
    best_trial_id = np.argmin(val_losses)
    best_trial_str = "trial-" + str(best_trial_id)
    best_trial_dir = os.path.join(test_unit_dir, best_trial_str)
    predictions_name = "_".join([
        dataset_name,
        model_name,
        "test-" + test_unit_str,
        best_trial_str,
        "predict-" + test_unit_str,
        "clip-predictions.csv"
    ])
    print(test_unit_str, best_trial_str)
    prediction_path = os.path.join(
        best_trial_dir, predictions_name)

    # Load prediction.
    with open(prediction_path, 'r') as f:
        reader = csv.reader(f)
        rows = list(reader)
    rows = [",".join(row) for row in rows]
    rows = rows[1:]
    rows = "\n".join(rows)

    # Parse rows with correct header.
    df = pd.read_csv(StringIO(rows),
        names=[
            "Dataset",
            "Test unit",
            "Prediction unit",
            "Timestamp",
            "Center Freq (Hz)",
            "Augmentation",
            "Key",
            "Ground truth",
            "Predicted probability"])
    y_pred = np.array(df["Predicted probability"])
    y_pred = (y_pred > 0.5).astype('int')

    # Load ground truth.
    y_true = np.array(df["Ground truth"])

    # Compute confusion matrix.
    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(
        y_true, y_pred).ravel()

    acc = 100 * (tn+tp) / (tn+tp+fn+fp)
    print(acc)
    
    tns.append(tn)
    fps.append(fp)
    fns.append(fn)
    tps.append(tp)
    accs.append(acc)

fn = sum(fns)
fp = sum(fps)
tn = sum(tns)
tp = sum(tps)
acc = np.mean(accs)

print("")
print(acc)
print(100 * (tn+tp) / (tn+tp+fn+fp))

unit01 trial-3
52.9049897471
unit02 trial-9
93.488372093
unit03 trial-2
99.1001865467
unit05 trial-8
96.5805471125
unit07 trial-3
93.4915773354
unit10 trial-2
97.2949261588

88.8100998322
92.7744195243


In [12]:
# Define model directory
model_name = "pcen-convnet"
aug_kind_str = "none"
if not aug_kind_str == "none":
    model_name = "_".join([model_name, "aug-" + aug_kind_str])
model_dir = os.path.join(models_dir, model_name)

fns, fps, tns, tps = [], [], [], []

test_unit_id = 0

test_unit_str = units[test_unit_id]
test_unit_dir = os.path.join(model_dir, test_unit_str)

for trial_id in range(n_trials):
    trial_str = "trial-" + str(trial_id)
    trial_dir = os.path.join(test_unit_dir, trial_str)
    predictions_name = "_".join([
        dataset_name,
        model_name,
        "test-" + test_unit_str,
        trial_str,
        "predict-" + test_unit_str,
        "clip-predictions.csv"
    ])
    
    prediction_path = os.path.join(
        trial_dir, predictions_name)

    # Load prediction.
    try:
        with open(prediction_path, 'r') as f:
            reader = csv.reader(f)
            rows = list(reader)
        rows = [",".join(row) for row in rows]
        rows = rows[1:]
        rows = "\n".join(rows)

        # Parse rows with correct header.
        df = pd.read_csv(StringIO(rows),
            names=[
                "Dataset",
                "Test unit",
                "Prediction unit",
                "Timestamp",
                "Center Freq (Hz)",
                "Augmentation",
                "Key",
                "Ground truth",
                "Predicted probability"])
        y_pred = np.array(df["Predicted probability"])
        y_pred = (y_pred > 0.5).astype('int')

        # Load ground truth.
        y_true = np.array(df["Ground truth"])

        # Compute confusion matrix.
        tn, fp, fn, tp = sklearn.metrics.confusion_matrix(
            y_true, y_pred).ravel()
        print(100 * (tn+tp) / (tn+tp+fn+fp))
    except:
        pass

74.5727956254
75.5126452495
73.8209159262


49.780000000000001